In [1]:
import json
from random import shuffle

In [2]:
with open('data_human_rights.json') as json_posFile:
    posData = json.load(json_posFile)

In [3]:
print len(posData)
print posData[0]
x = posData[0]
len(x['categories'])

65630
{u'url': u'https://www.business-humanrights.org/finally-wto-to-open-debate-on-patents-and-low-cost-drugs', u'body': u"The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", u'companies': [], u'categories': [u'Pharmaceutical', u'Health: General (including workplace health & safety)', u'Access to medicines', u'Intellectual property', u'WTO'], u'title': u'Finally, WTO to Open Debate on Patents and Low-Cost Drugs'}


5

In [4]:
import re
from w3lib.html import remove_tags
def clense(text, space_replacer = ' ', to_lower = True, remove_punc = True):
    # remove HTML comments first as suggested in https://stackoverflow.com/questions/28208186/how-to-remove-html-comments-using-regex-in-python
    text = re.sub("(<!--.*?-->)", "", text, flags=re.DOTALL)
    text = remove_tags(text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)   #remove non-ascii characters
    text = text.replace("&amp;", "and")
    text = text.replace("&", "and")
    text.strip()
    text.rstrip()
    text = text.replace("\r\n", "")
    text = text.replace("\n", "")
    text = text.replace("\"", "")
    if to_lower:
        text = text.lower()

    if remove_punc:
        # from https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        text = re.sub(r'[^\w\s]', '', text)   #remove punctuation marks and non-word
        text = text.replace(",", "")

    text = re.sub(' +', space_replacer, text)
    #if  all(ord(char) < 128 for char in text) == False:
    #    text = ''
    ''.join(i for i in text if ord(i)<128)
    return text

In [27]:
# one label for each body
new_posData = []
for x in posData:
    if x['body'] is not None:
        if len(x['body']) < 5: # if body is empty
            continue
        for y in range (len(x['categories'])):
            each_data = {}
            each_data['body'] = x['body']
            unitemp = each_data['body'].encode('utf8')
            each_data['body'] = clense(unitemp,to_lower=False,remove_punc=False).strip()
            each_data['category'] = []
            each_data['category'].append(x['categories'][y])
            new_posData.append(each_data)

In [28]:
print new_posData[0]
print new_posData[1]
print new_posData[2]

{'body': u"The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", 'category': [u'Pharmaceutical']}
{'body': u"The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", 'category': [u'Health: General (including workplace health & safety)']}
{'body': u"The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their ar

In [5]:
# multi labels for each body
new_posData = []
for x in posData:
    each_data = {}
    if x['body'] is not None:
        if len(x['body']) < 5: # if body is empty
            continue
        each_data['body'] = x['body']
        unitemp = each_data['body'].encode('utf8')
        each_data['body'] = clense(unitemp,to_lower=False,remove_punc=False).strip()
        each_data['category'] = []
        for y in range (len(x['categories'])):
            each_data['category'].append(x['categories'][y])
        new_posData.append(each_data)

In [6]:
shuffle(new_posData)

In [50]:
# single data trial
print new_posData[0]
y = new_posData[0]['category'][1]
print y
label = ''
cat = ''
for i in y:
    if i == ' ':
        cat = cat + '_'
    else :
        cat = cat + i
label = label + '__label__' + cat + '__  '
print cat   
print label
label = label + '__label__' + cat + '__  '
print label
print label + new_posData[0]['body']

{'body': u"The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", 'category': [u'Pharmaceutical', u'Health: General (including workplace health & safety)', u'Access to medicines', u'Intellectual property', u'WTO']}
Health: General (including workplace health & safety)
Health:_General_(including_workplace_health_&_safety)
__label__Health:_General_(including_workplace_health_&_safety)__  
__label__Health:_General_(including_workplace_health_&_safety)__  __label__Health:_General_(including_workplace_health_&_safety)__  
__label__Health:_General_(including_workplace_health_&_safety)__  __label__Health:_General_(including_workplace_health_&_safety)__  The much-awaited debate on how international patent laws affect developing countries' access to medications b

In [7]:
f = open("leaf_bhr.txt","w+")
for x in new_posData:
    label = ""
    body = x['body'].encode('utf-8').strip()
    for y in x['category']:
        cat = ''
        for i in y:
            if i == ' ':
                cat = cat + '_'
            else :
                cat = cat + i
        label = label + '__label__' + cat.encode('utf-8').strip() + '__  '        
    f.write(label + body + '\n')
f.close()